In [1]:
%reload_ext autoreload
%autoreload 2
from vktest.dataset import VCTK
from vktest.litmodel import LitAgainVC
from torch.utils.data import DataLoader

import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint

import wandb
from pytorch_lightning.loggers import WandbLogger

In [2]:
root = "../dataset/"
VCTK_train = VCTK(root, train=True)
VCTK_val = VCTK(root, train=False)

In [3]:
train_loader = DataLoader(dataset=VCTK_train, num_workers=4, batch_size=100, pin_memory=True)
val_loader = DataLoader(dataset=VCTK_val, num_workers=4, batch_size=100, pin_memory=True)

In [4]:
encoder_params = {
    "melbins": 80,
    "melbins_out": 4,
    "hidden_size": 256,
    "conv_blocks_num": 6
}
decoder_params = {
    "melbins": 4,
    "melbins_out": 80,
    "hidden_size": 256,
    "conv_blocks_num": 6
}
model = LitAgainVC(encoder_params, decoder_params)

In [5]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: zurikz (use `wandb login --relogin` to force relogin)


True

In [6]:
checkpoint_callback = ModelCheckpoint(dirpath='.')

wandb_logger = WandbLogger(
    name="again_vc_training",
    project="vktest"
)

trainer = Trainer(
    gpus=1,
    callbacks=[checkpoint_callback],
    log_every_n_steps=10,
#     precision=16,
#     overfit_batches=0.01,
    max_steps=100000,
#     tpu_cores=8,
#     default_root_dir="",
    weights_save_path="checkpoints",
    gradient_clip_val=3,
    benchmark=True,
    auto_scale_batch_size='binsearch',
    logger=wandb_logger,
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [ ]:
trainer.fit(
    model=model,
    train_dataloader=train_loader,
    val_dataloaders=val_loader
)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



  | Name    | Type          | Params
------------------------------------------
0 | encoder | Encoder       | 416 K 
1 | decoder | Decoder       | 1.2 M 
2 | sigmoid | CustomSigmoid | 0     
------------------------------------------
1.6 M     Trainable params
0         Non-trainable params
1.6 M     Total params
6.486     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]